# Исследование надежности заемщиков.

Целью данной работы является анализ заемщиков на предмет их надежности для заемодателя.

**Необходимо проверить несколько гипотез касательно этого вопроса:**
 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 4. Как разные цели кредита влияют на его возврат в срок?
 
Все данные находятся в фале data.csv, о качестве данных сведений нет.

**В ходе выполнения работы будет необходимо провести:**
 1. обзор данных предоставленных заказчиком;
 2. предобработку данных;
 3. проверить необходимые гипотезы.
 
**Описание данных:**
 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

**Оглавление**

1. [Шаг 1. Обзор данных](#start)
2. [Шаг 2. Предобработка данных](#preprocessing)
    * [Шаг 2.1 Заполнение пропусков, замена типов данных](#2.1)
    * [Шаг 2.2 Удаление дубликатов.](#2.2)
    * [Шаг 2.3. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.](#2.3)
    * [Шаг 2.4. Категоризация дохода.](#2.4)
    * [Шаг 2.5. Категоризация целей кредита.](#2.5)
3. [Шаг 3. Проверка гипотез](#test)
    * [Гипотеза 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?](#3.1)
    * [Гипотеза 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?](#3.2)
    * [Гипотеза 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#3.3)
    * [Гипотеза 4: Как разные цели кредита влияют на его возврат в срок?](#3.4)
4. [Общий вывод](#finish)

## <a id="start">Шаг 1. Обзор данных</a>

Первым шагом рассмотрим общую информацию о данных предоставленных для анализа

In [1]:
import pandas as pd # импорт библиотеки pandas
from IPython.display import display # импортируем метод display из библиотеки IPython.display
df = pd.read_csv('/datasets/data.csv')
df.describe() # просмотр статистических характеристик таблицы

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


При просмотре статистических данных таблицы в двух столбцах(children и days_employed) обнаружены отрицательные значения, их появление может быть связано с особенностями ПО из которого были получены данные.

В столбце children(количество детей) имеется максимальное значение = 20, объяснить можно двумя причинами:
 1. Данные подавались семейными детскими домами, в этом случае можно принять такое количество детей в семье;
 2. При вводе данных была допущена ошибка в следствии человеческого фактора.
 
В том и в другом случае необходимо запросить уточнение данной информации.

In [3]:
df.info() # просмотр общей информации о типах данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# просматриваем уникальные значения в текстовых столбцах документа
display(df['education'].unique())
display(df['family_status'].unique())
display(df['gender'].unique())
display(df['income_type'].unique())
display(df['purpose'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['F', 'M', 'XNA'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<ul>
    <li>в столбце children имеются отрицательныйе значения, которые необходимо заменить на положительные</li>
    <li>в столбце days_employed есть пропуски и отрицательные значения, тип float нужно заменить на int</li>
    <li>в столбце total_income заполнить пропуски и заменить float на int</li>
    <li>привести строковые столбцы к одному регистру</li>
    <li>значения столбца purpose объеденить в единые смысловые категории</li>
    <li>обработать дубликаты</li>
</ul>

In [5]:
# приводим текстовые значения столбцов таблицы в нижний регстр.
df['education'] = df['education'].str.lower() 
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

In [6]:
# преобразуем отрицательные значения в положительные
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()

## <a id="preprocessing">Шаг 2. Предобработка данных</a>

### <a id="2.1">Шаг 2.1 Заполнение пропусков, замена типов данных</a>

In [7]:
df.isna().sum() # поиск и подсчет пропусков в таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
# проверим, находятся ли пропуски в одинаковых строках, т.к. их количество совпадает
df[(df.days_employed.isna()) & (df.total_income.isna())] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,m,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,m,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,f,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,f,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,m,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,f,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,f,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,f,сотрудник,0,NaN,строительство жилой недвижимости


После просмотра пропущенных значений можно сделать следующие выводы:
 1. Нахождение пропусков в одних и тех же строках может говорить о неофициальной трудовой деятельности клиента
    из-за чего установить реальные показатель стажа и зароботка не представляется возможным;
 2. Пропуски могли возникнуть из-за ошибки при выгрузке данных, в этом случае необходимо обратиться к человеку ответственному за     эту операцию и запросить повторную выгрузку и уточнение данных.
 
Исходя из вышеописанного и целей анализа можно утверждать, что заполнение этих данных медианным значением не имеет смысла, т.к. в случае с доходом это может привести к искажению конечных результатов анализа, а в случае со столбцом days_employed просто не имеет смысла, т.к. эти данные не будут учавствоваться в проверке необходимых гипотез.

По этому пропущенные значения будут заменены на нулевые.

In [9]:
# проводим замену значений NAN на медианное и проверяем все ли получилось
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
# преобразуем значения float к значениям типа int
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [12]:
# перенес это изменение из конца сюда, закоментил эту часть в выводах
df.loc[df['children'] == 20, 'children'] = 2

### <a id="2.2">Шаг 2.2 Удаление дубликатов.</a>

Без дубликатов в данных не обойтись, найдем их.

In [13]:
df.duplicated().sum()

71

При общем количестве строк (более 20 тыс.) 71 дубликат не сыграет роли в итогах анализа, дубликаты можно просто удалить.

In [14]:
# Удаляем дубликаты, исправляем индексы и проверяем все ли получилось
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [15]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,60570.114617,43.271231,0.817097,0.973898,0.081150,1.652256e+05
std,0.756069,133435.760575,12.570822,0.548674,1.421567,0.273072,9.802102e+04
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1023.000000,33.000000,1.000000,0.000000,0.000000,1.076230e+05
50%,0.000000,2194.000000,42.000000,1.000000,0.000000,0.000000,1.450170e+05
75%,1.000000,4797.000000,53.000000,1.000000,1.000000,0.000000,1.958132e+05
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [16]:
display(df['education'].unique())
display(df['family_status'].unique())
display(df['gender'].unique())
display(df['income_type'].unique())
display(df['purpose'].unique())

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

array(['f', 'm', 'xna'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из аномалий остались только те, которые не повлияют на результат исследования:
 - в столбце gender присутствует значение "xna", исправить это значение на достоверное не представляется возможным;
 - в столбце days_employed присутствуют слишком большие значения, но это информация не используется в исследовании.

### <a id="2.3">Шаг 2.3. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.</a>

Для упрощения дальнейшей работы с данными создадим два дата фрейма на основании слобцов таблицы, где показателям присвоены сооветствующие ID. Это пригодтся для составления итоговых сводных таблиц. Обращение по ID предпочтительней, чем группировка всех данных по значениям.

In [17]:
# Создаем дата фрейм из двух столбцов основной таблицы
df_education = df[['education', 'education_id']]
# Удаляем дубликаты, оставив краткую таблицу со значениями и ID
df_education = df_education.drop_duplicates().reset_index(drop=True)
df_education # Выводим на экран новый дата фрейм

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


По аналогии с первым дата фреймом создадим второй

In [18]:
df_family = df[['family_status', 'family_status_id']]
df_family = df_family.drop_duplicates().reset_index(drop=True)
df_family

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [19]:
# Удалим из основной таблицы ненужные столбцы
del df['education']
del df['family_status']

In [20]:
df.head() # Для проверки выведем на экран основную таблицу

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,f,пенсионер,0,158616,сыграть свадьбу


### <a id="2.4">Шаг 2.4. Категоризация дохода.</a>

В столбце "общий доход" (total_income) содержится много информации с которой не удобно работать. Для удобства нужно разделить ее на диапазоны и создать категорию для каждого из них.

Для этой задачи создадим функцию, которая просматривает столбкц total_income и сравнивает каждое значение с условиями диапазонов. На основании полученных данных создадим столбец total_income_category и добавим его к общей таблице методом .apply()

In [21]:
# Создаем функцию для просмотра исходных значений и определения их категорий
def income_category(total_income):
    if total_income >= 0 and total_income < 30001:
        return 'E'
    if total_income > 30000 and total_income < 50001:
        return 'D'
    if total_income > 50000 and total_income < 200001:
        return 'C'
    if total_income > 200000 and total_income < 1000001:
        return 'B'
    if total_income > 1000000:
        return 'A'

# Создаем новый столбец содержащий информацию о категориях уровня дохода и присоеденяем его к таблице
df['total_income_category'] = df['total_income'].apply(income_category)
df.head(10) # Для проверки выводим таблицу на экран

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,m,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,f,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,m,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C


In [22]:
df.isna().sum() # Проверим дополненную таблицу на предмет появления значений NaN

children                 0
days_employed            0
dob_years                0
education_id             0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
dtype: int64

In [23]:
# проверим все ли получилось
df['total_income_category'].value_counts()

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### <a id="2.5">Шаг 2.5. Категоризация целей кредита.</a>

Аналогично информации о доходах клиентов небходимо категоризировать информацию о целях кредита.

Создаем функцию, которая будет искать в строках совпадения по ключевым частям слов, найденные значения разделим на заранее определенные категории. Сформируем новый столбец с категориями на основании данных полученных из функции и присоеденим его к основной таблице.

In [24]:
def category_for_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
    if 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'


df['purpose_category'] = df['purpose'].apply(category_for_purpose)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,m,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,f,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,m,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [25]:
df.isna().sum() # Проверим дополненную таблицу на предмет появления значений NaN

children                 0
days_employed            0
dob_years                0
education_id             0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
purpose_category         0
dtype: int64

In [26]:
df['purpose_category'].unique() # просмотр уникальных значений

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

## <a id="test">Шаг 3. Проверка гипотез</a>

К данному этапу мы полностью обработали необходимую информацию. Теперь пришло время отвечать на главные вопросы, которые поставил заказчик.

Для подсчета процента возможности просрочки по кредиту во всех таблицах этого раздела будет использоваться столбец 'debt' содержащий индекс наличия задолженности по кредиту (0 - задолженности нет, 1 - задолженность есть).

Приступим!

### <a id="3.1">Гипотеза 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?</a>

С помощью метода .pivot_table() создадим сводную таблицу, основным будет столбец содержащий информацию о количестве детей у заемщика(children), по какому столбцу будет вестись подсчет в данном случае не принципиально.

Далее будет создан столбец pay_index в котором будет находиться значение вероятности возникновения задолженности по кредиту в процентах.

In [27]:
# Создаем сводную таблицу методом .pivot_table
pay_with_children = df.pivot_table(index = 'children', columns = 'debt', values = 'education_id', 
                                    aggfunc='count', fill_value=0)

# Проводим расчет вероятности возникновения задолженности
# прикрепляем столбец с этими сведениями к сводной таблице
pay_with_children['pay_index'] = ((pay_with_children[1] / pay_with_children[0]) * 100).round(2).astype(str) + '%'
pay_with_children.sort_values(by = 'pay_index') # Выводим на экран сводную таблицу, сортируем по убыванию

debt,0,1,pay_index
children,,,
5,9,0,0.0%
1,4410,445,10.09%
2,1926,202,10.49%
4,37,4,10.81%
0,13028,1063,8.16%
3,303,27,8.91%


##### Вывод:

Проверка гипотезы показала, что количество детей у заемщика может отклонить вероятность просрочки максимум на 3% в большую сторону.

В данном случае выделяются заемщики с 5 детьми, у них просрочка отсутствует полностью, но т.к. их всего 9 человек, то этот параметр не стоит принимать в серьез, для его проверки нужна более обширная выборка.

 - Самыми надежными клиентами можно назвать тех, у кого детей нет или у кого 3 ребенка;
 - Клиенты с 1, 2 и 4 детьми допускаю просрочку по кредиту с вероятностью, в среднем, 10.4%, думаю с ними нужно быть внимательнее и     проводить более тщательную проверку.

### <a id="3.2">Гипотеза 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?</a>

Создаем сводную таблицу по аналогии с первой, основной столбец заменяем на family_status_id, т.к. необходимо узнать возможность возникновения долга в зависимости от семейного статуса.

In [29]:
pay_family_status = df.pivot_table(index = 'family_status_id', columns = 'debt', values = 'education_id', 
                                    aggfunc='count', fill_value=0)

# С помощью метода .merge() присоеденяем таблицу содержащую семейный статус
# "склеивать" таблицы будем по столбцу содержащему ID семейного статуса
pay_family_status = df_family.merge(pay_family_status, on='family_status_id', how='left')
# Для того что бы избежать путанницу из дополненной таблицы удаляем столбец содержащий ID семейного положения
del pay_family_status['family_status_id']

pay_family_status['pay_index'] = ((pay_family_status[1] / pay_family_status[0]) * 100).round(2).astype(str) + '%'
pay_family_status.sort_values(by = 'pay_index')

,family_status,0,1,pay_index
1,гражданский брак,3763,388,10.31%
4,не женат / не замужем,2536,274,10.8%
2,вдовец / вдова,896,63,7.03%
3,в разводе,1110,85,7.66%
0,женат / замужем,11408,931,8.16%


##### Вывод:

Проверка данной гипотезы показала интересные закономерности:
 - Самыми безответственными в отношении погашения кредитов оказались люди либо холостые, либо состоящие в незарегестрированных отношениях;
 - Лучшие показатели показывают люди, чьи отношения по тем или иным причинам прекратились, думаю ответ на вопрос "почему" лежит уже скорее в области психологии;
 - Средним результатом могут похвастаться пары состоящие в официально зарегистрированных отношениях.
 
Думаю, что категорию "холостяков" следует подвергать более внимательному анализу на предмет надежности источников дохода и положительной кредитной истории.

### <a id="3.3">Гипотеза 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</a>

Создаем сводную таблицу по аналогии с первой, основной столбец заменяем на total_income_category, т.к. необходимо узнать возможность возникновения долга в зависимости от дохода.

In [30]:
pay_income_status = df.pivot_table(index = 'total_income_category', columns = 'debt', values = 'education_id', 
                                    aggfunc='count', fill_value=0)

pay_income_status['pay_index'] = ((pay_income_status[1] / pay_income_status[0]) * 100).round(2).astype(str) + '%'
pay_income_status.sort_values(by = 'pay_index')

debt,0,1,pay_index
total_income_category,,,
E,20,2,10.0%
D,329,21,6.38%
B,4685,356,7.6%
A,23,2,8.7%
C,14656,1360,9.28%


##### Вывод:

Рассмотрим результаты гипотезы зависимости между уровнем дохода и возвратом займа в срок:
 - Явно выделяется только группа с уровнем доходв 'D' (до 30 тыс.руб.), с показателем 6.38% они показываю себя самыми ответственными плательщиками;
 - Группы категорий 'C' (от 50 до 200 тыс.руб.) и 'Е' (от 30 до 50 тыс.руб.) имеют самые большие показатели вероятности просрочки кредита составляющие 9.28% и 10% соответственно, думаю стоит тщательнее их проверять;
 - Остальные группы заемщиков показывают средний результат.

### <a id="3.4">Гипотеза 4: Как разные цели кредита влияют на его возврат в срок?</a>

Создаем сводную таблицу по аналогии с первой, основной столбец заменяем на purpose_category, т.к. необходимо узнать возможность возникновения долга в зависимости от цели кредита.

In [31]:
pay_purpose_status = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'education_id', 
                                    aggfunc='count', fill_value=0)

pay_purpose_status['pay_index'] = ((pay_purpose_status[1] / pay_purpose_status[0]) * 100).round(2).astype(str) + '%'
pay_purpose_status.sort_values(by = 'pay_index')

debt,0,1,pay_index
purpose_category,,,
получение образования,3643,370,10.16%
операции с автомобилем,3903,403,10.33%
операции с недвижимостью,10029,782,7.8%
проведение свадьбы,2138,186,8.7%


##### Вывод:

Рассмотрим результаты гипотезы влияния целей кредита на его возврат в срок:
 - Самые лучшие показатели показывают самые счастливые люди, покупка недвижимости или свадьба похоже прибаляет ответственности, показатели 7.8% и 8.7% соответственно являются минимальными из всех частовстречаемых в данном исследовании;
 - Кредиты взятые на получение образования и операции с автомобилями имеют больший процент вероятности просрочки займа, считаю что это связано:
   - у первых с проблемами в трудоустройстве;
   - у вторых из-за не просчитанных затрат на содержание нового автомобиля.

## <a id="finish">Общий вывод</a>

По итогам исследования можно сказать, что за редким исключением вероятность просрочки возврата займа находится в диапазоне от 7.5% до 10.5%.

Следует уделять особое внимание клиентам со средним уровнем дохода, имеющим детей, находящимся в незарегистрированных отнашениях или берущим займ на цели связанные с образованием или на операции с автотранспортом. Для этих клиентов необходимо ввести более жесткую систему контроля проверки информации.

Можно дать некоторые рекомендации банку:
 1. Провести работу по улучшению качества информации о клиентах;
 2. Исключить ручнойй ввод информации о клиентах сотрудниками банка, ввести систему для шаблонизации информации;
 3. Более тщательно контролировать выгрузку данных;
 4. При обнаружении недочетов в информации о клиентах организовать службу, которая будет связываться с клиентом и уточнять данные;
 5. Провести работы по оптимизации ПО предназначенного для хранения информации;
 6. Предлагаю отсеивать клиентов у котрых не возможно проверить достоверность информации о доходах.